# CELL TOWER MAXIMALCOVERAGE AT MINIMAL COST

## Problem Description

A telecommunications company needs to build a set of mobile phone towers to provide signal coverage to the inhabitants of a given city. Several potential locations have been identified where the towers could be built. The towers have a fixed range and due to budget restrictions, only a limited number of them can be built. Given these restrictions, the company wants to provide coverage to the largest possible percentage of the population. To simplify the problem, the company has divided the area it wants to cover into a set of regions, each of which has a known population. The objective is then to choose in which of the potential locations the company should build cell phone towers, to provide coverage to as many people as possible. 

### RANDOM DATA GENERATION

In [32]:
import numpy as np

number_of_towers = 11
number_of_regions = 7

tower_coverage = np.random.randint(0, 2, size=(number_of_towers, number_of_regions))
population_region = np.random.randint(1, 100, size=(number_of_regions))
tower_cost = np.random.randint(1, 100, size=(number_of_towers))
budget = np.random.randint(sum(tower_cost) // 2, sum(tower_cost))

### TOWER COVERAGE BY REGION

In [33]:
print(tower_coverage)

[[0 1 0 0 1 0 1]
 [1 0 1 0 0 1 1]
 [0 0 1 1 0 1 0]
 [0 0 0 1 1 1 1]
 [1 1 0 0 1 0 1]
 [1 0 0 0 0 1 0]
 [0 1 0 1 0 1 1]
 [0 1 1 1 1 1 0]
 [0 1 1 0 1 1 1]
 [0 1 0 0 0 0 1]
 [1 1 1 0 0 1 0]]


### POPULATION BY REGION

In [34]:
print(population_region)

[97 77 52 88  2 35 93]


### TOWER COST

In [35]:
print(tower_cost)

[27 90 46 47  1 61 48 32 54 51 14]


### BUDGET

In [36]:
print(budget)

326


# PEQNP TENSOR MODEL

In [37]:
!pip install peqnp
import peqnp as cnf
import functools
import operator

cost = budget
while True:
    optimal = 0
    X_opt = None
    while True:

        cnf.engine(10)

        X = cnf.tensor(dimensions=(number_of_towers, number_of_regions))

        # ensure that at least one tower that covers a region must be selected.
        for i in range(number_of_towers):
            assert sum(X[[i, j]](0, 1) for j in range(number_of_regions)) <= 1

        # ensure that at least one tower that covers a region must be selected.
        for j in range(number_of_regions):
            assert sum(X[[i, j]](0, 1) for i in range(number_of_towers)) >= 1

        # ensure that the total cost of building towers do not exceed the allocated budget.
        assert sum(functools.reduce(operator.ior, [X[[i, j]](0, 1) for j in range(number_of_regions)]) * tower_cost[i] for i in range(number_of_towers)) <= cost


        # We seek to maximize the total population covered by the towers.
        assert sum(X[[i, j]](0, population_region[j] * tower_coverage[i, j]) for i in range(number_of_towers) for j in range(number_of_regions)) > optimal

        if cnf.satisfy(turbo=True):
            X_opt = np.vectorize(int)(X.binary)
            optimal = sum(X_opt[i][j] * population_region[j] * tower_coverage[i, j] for i in range(number_of_towers) for j in range(number_of_regions))        
            print(optimal)
        else:
            if X_opt is None:
                print('Infeasible...')   
            else:
                optimal = sum(X_opt[i][j] * population_region[j] * tower_coverage[i, j] for i in range(number_of_towers) for j in range(number_of_regions))        

                cost = 0
                for i in range(number_of_towers):
                    if sum(X_opt[i]) > 0:
                        cost += tower_cost[i]
                
                covering = 0
                for j in range(number_of_regions):
                    if sum(X_opt[i][j] for i in range(number_of_towers)) > 0:
                        covering += population_region[j]

                print(80 * '-')
                print('COST     vs BUDGET      : {} vs {}'.format(cost, budget))
                print('COVERING vd OVER COVERING vs POPULATION  : {} vs {} vs {}'.format(covering, optimal, sum(population_region)))
                print('TOWERS:')
                for i in range(number_of_towers):
                    if sum(X_opt[i]) > 0:
                        print('TOWER Nº {}'.format(i + 1))
                print(80 * '-')
                print(X_opt)
                cost -= 1
            break 
    if X_opt is None:
        print('OPTIMAL!') 
        break 

392
477
485
494
521
532
541
544
565
607
625
634
--------------------------------------------------------------------------------
COST     vs BUDGET      : 320 vs 326
COVERING vd OVER COVERING vs POPULATION  : 444 vs 634 vs 444
TOWERS:
TOWER Nº 1
TOWER Nº 3
TOWER Nº 4
TOWER Nº 5
TOWER Nº 7
TOWER Nº 8
TOWER Nº 9
TOWER Nº 10
TOWER Nº 11
--------------------------------------------------------------------------------
[[0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0]]
444
480
535
539
541
--------------------------------------------------------------------------------
COST     vs BUDGET      : 318 vs 326
COVERING vd OVER COVERING vs POPULATION  : 444 vs 541 vs 444
TOWERS:
TOWER Nº 1
TOWER Nº 2
TOWER Nº 3
TOWER Nº 4
TOWER Nº 5
TOWER Nº 6
TOWER Nº 8
TOWER Nº 11
--------------------------------------------------------------------------------
[[0 1 0 0 0 0 0]
 

#### Copyright © 2021 PEQNP